In [210]:
import pandas as pd
import numpy as np
import pandas_datareader.data as wb
import matplotlib.pyplot as plt
import string
import warnings
warnings.filterwarnings('ignore')
import requests

In [211]:
acao_codigo = 'FLRY3'
menor_MM = []
maior_MM = []

In [212]:
def format_money(m):
    return f'R$ {m:.2f}'

In [213]:
for m in range(100, 301, 10):
    mm2 = ('MM'+str(m), m)
    print(f'\n{mm2[0]:=^120}')
    for j in range(1, 31):
        mm1 = ('MM'+str(j), j)
        print(f'{mm1[0]}', end='.')
        
        var_alta = 500
        var_queda = 20

        dados = pd.DataFrame()
        dados[acao_codigo] = wb.get_data_yahoo(acao_codigo+'.SA', start=2000)['Adj Close']
            
        dados[mm1[0]] = dados[acao_codigo].rolling(mm1[1]).mean()
        dados[mm2[0]] = dados[acao_codigo].rolling(mm2[1]).mean()
        dados = dados.dropna()


        lucro, h = 0, 0
        compra, vende = 0, 0
        stopAtivado = False
        precoStopAtivado = 0
        for i in dados.index:
            preco_acao = dados[acao_codigo][i]
            dia = f'{i.year}/{i.month:02}/{i.day:02}'
            # Comprando
            if dados[mm1[0]][i] >= dados[mm2[0]][i] and dados[mm1[0]][h] <= dados[mm2[0]][h] and compra == 0:
                compra = preco_acao
                #print(f'{dia} - Compra acões à {format_money(compra)}...')
                stop_acao = preco_acao*(1-(var_queda/100))
                alta_acao = preco_acao*(1+(var_alta/100))
            elif stopAtivado and dados[mm1[0]][i] >= dados[mm2[0]][i] and precoStopAtivado <= preco_acao and compra == 0:
                compra = preco_acao
                #print(f'{dia} - Compra acões à {format_money(compra)}...')
                stop_acao = preco_acao*(1-(var_queda/(100*2)))
                alta_acao = preco_acao*(1+(var_alta/(100*2)))
                stopAtivado = False

            # Vendendo
            elif compra != 0:
                menor_preco_acao = preco_acao*(1-(var/100))
                if menor_preco_acao > stop_acao:
                    stop_acao = menor_preco_acao
                    #print(f'{dia} - Stop da açao foi para {format_money(stop_acao)}')
                elif dados[mm1[0]][i] <= dados[mm2[0]][i] and dados[mm1[0]][h] >= dados[mm2[0]][h] or preco_acao < stop_acao or preco_acao >= alta_acao:
                    if preco_acao < stop_acao:
                        stopAtivado = True
                        precoStopAtivado = preco_acao
                        #print(f"{'Foi stopado!':>60}")
                    vende = preco_acao
                    #print(f'{dia} - Vende ações à {format_money(vende)}...')
                    luc_parcial = (vende - compra)
                    #if luc_parcial > 0:
                        #print(f';D Lucro de: {format_money(luc_parcial)}...\n')
                    #else:
                        #print(f':( Preju de: {format_money(luc_parcial)}...\n')
                    lucro += luc_parcial
                    compra, vende = 0, 0
            elif dados[mm1[0]][i] <= dados[mm2[0]][i] and dados[mm1[0]][h] >= dados[mm2[0]][h]:
                stopAtivado = False
                precoStopAtivado = 9999999
            h = i

        if compra != 0:
            vende = dados[acao_codigo][-1]
            #print(f'{dia} - Vende ações à {format_money(vende)}...')
            luc_parcial = (vende - compra)
            #if luc_parcial > 0:
            # print(f';D Lucro de: {format_money(luc_parcial)}...\n')
            #else:
            # print(f':( Preju de: {format_money(luc_parcial)}...\n')
            lucro += luc_parcial

        #print(f'Lucro total: {format_money(lucro)}')
        primeira_acao = dados[acao_codigo][0]
        lucr_porc = lucro/primeira_acao*100
        #print(f'{lucr_porc:.2f}% de lucro sobre a primeira ação de {format_money(primeira_acao)}')
        menor_MM.append([j, round(lucr_porc, 2)])
    menor_MM = sorted(menor_MM, key=lambda x: x[1], reverse=True)
    maior_MM.append([mm2[0], [menor_MM]])
    menor_MM = []


=========================================================MM100==========================================================
MM1.MM2.MM3.MM4.MM5.MM6.MM7.MM8.MM9.MM10.MM11.MM12.MM13.MM14.MM15.MM16.MM17.MM18.MM19.MM20.MM21.MM22.MM23.MM24.MM25.MM26.MM27.MM28.MM29.MM30.
=========================================================MM110==========================================================
MM1.MM2.MM3.MM4.MM5.MM6.MM7.MM8.MM9.MM10.MM11.MM12.MM13.MM14.MM15.MM16.MM17.MM18.MM19.MM20.MM21.MM22.MM23.MM24.MM25.MM26.MM27.MM28.MM29.MM30.
=========================================================MM120==========================================================
MM1.MM2.MM3.MM4.MM5.MM6.MM7.MM8.MM9.MM10.MM11.MM12.MM13.MM14.MM15.MM16.MM17.MM18.MM19.MM20.MM21.MM22.MM23.MM24.MM25.MM26.MM27.MM28.MM29.MM30.
=========================================================MM130==========================================================
MM1.MM2.MM3.MM4.MM5.MM6.MM7.MM8.MM9.MM10.MM11.MM12.MM13.MM14.MM15.MM16.MM17.MM18.MM19.MM2

In [214]:
x=0
for i in maior_MM:
    print(i)
    x += 1

['MM100', [[[11, 272.38], [17, 255.13], [12, 254.59], [16, 245.62], [15, 244.97], [13, 241.09], [18, 239.7], [14, 239.15], [10, 230.96], [9, 225.69], [7, 225.05], [22, 214.37], [19, 214.05], [8, 213.61], [3, 213.13], [20, 210.78], [29, 205.32], [24, 202.81], [26, 194.59], [21, 194.53], [27, 192.47], [25, 192.41], [28, 192.3], [2, 187.55], [23, 187.19], [4, 186.39], [6, 180.38], [5, 180.18], [30, 177.17], [1, 91.34]]]]
['MM110', [[[12, 246.77], [14, 239.44], [3, 239.21], [16, 232.85], [13, 230.84], [17, 226.87], [15, 221.17], [29, 213.19], [24, 210.8], [5, 209.07], [4, 208.63], [2, 206.88], [10, 203.72], [25, 202.48], [27, 201.7], [23, 197.81], [8, 194.5], [18, 193.63], [26, 192.81], [11, 191.59], [21, 190.94], [19, 189.86], [6, 189.64], [22, 189.43], [30, 189.15], [20, 189.06], [28, 187.58], [7, 184.39], [1, 184.03], [9, 182.32]]]]
['MM120', [[[12, 289.47], [13, 281.22], [11, 263.45], [19, 251.58], [3, 247.8], [14, 242.46], [2, 239.56], [8, 239.01], [15, 238.9], [6, 236.71], [24, 233.2

In [216]:

maior_MM[0][1][0][0]

[11, 272.38]

In [239]:
lista1 = []
for i in maior_MM:
    for j in i[1]:
        lista1.append(j[0][0])
print(sorted(lista1))

lista2 = []
for i in maior_MM:
    for j in i[1]:
        lista2.append(j[1][0])
print(sorted(lista2))

lista3 = []
for i in maior_MM:
    for j in i[1]:
        lista3.append(j[2][0])
print(sorted(lista3))

[3, 3, 3, 6, 7, 11, 12, 12, 17, 17, 18, 19, 20, 21, 22, 22, 22, 24, 25, 27, 27]
[2, 6, 12, 13, 13, 13, 13, 14, 14, 17, 18, 19, 19, 21, 22, 23, 24, 25, 26, 27, 30]
[3, 9, 9, 11, 12, 13, 13, 14, 14, 17, 19, 20, 21, 22, 23, 23, 23, 23, 23, 24, 24]
